# Optimization with Constraint Learning: A WFP case study

## The Palatable Diet Problem  

**Problem Description.** The Diet Problem is the first large-scale optimization problem to be solved with the Simplex algorithm by Jack Laderman in [1947](https://www.mpi-inf.mpg.de/fileadmin/inf/d1/teaching/winter18/Ideen/Materialien/Dantzig-Diet.pdf). The basic formulation of this problem consists of minimizing the cost of a food basket while meeting the specified nutrient requirements. In this notebook, we solve The Palatable Diet Problem (TPDP), where the basic model is extended with a constraint on the food basket palatability. An explicit formula of the palatability constraint is unknown, but we have data on several food baskets and the respective palatability score. First, we define a conceptual model with the *known constraint*. Then, OptiCL is used to learn and embed the palatability constraint.  
(*TPDP is part of a larger optimization problem which simultaneously  optimizes  the  food  basket  to  be  delivered,  the  sourcing  plan,  the  delivery  plan, and  the  transfer  modality  of  a  month-long  food  supply in a Wolrd Food Program setting ([Maragno et al., 2021])*).

**Objective function:** minimize the total cost of the food basket.  
$\min_{\boldsymbol{x}} c^\top \boldsymbol{x}$

*subject to* 

**Nutritional constraints:** for each nutrient $j\in\mathcal{N}$, at least meet the minimum required level.  
$ \sum_{k \in \mathcal{K}} nutval_{kj} x_{k} \geq nutreq_{j}, \ \ \ \forall l\in\mathcal{N},$   
**Constraints on sugar and salt.**</font>  
$ x_{salt} = 5,$   
$ x_{sugar} = 20,$  
**Palatability constraints:** the food basket palatability has to be at least equal to $t$.  
$ y \geq t,$  
**Learned predictive model:** the palatability is defined using a predictive model.  
$ y = \hat{h}(\boldsymbol{x}),$   
**Non negativity constraints.**  
$ x_{k} \geq 0, \ \ \ \forall k \in \mathcal{K}.$  

In [1]:
import pandas as pd
from imp import reload
import numpy as np
import math
from sklearn.utils.extmath import cartesian
import time
import sys
import os
import opticl
# import opticl.embed_mip as em
from pyomo import environ
from pyomo.environ import *

In [2]:
np.random.seed(0)

#### Auxiliary functions

In [3]:
def normalize(y):
    # Values based on the dataset before normalization
    minimum = 71.969 
    maximum = 444.847  
    return 1 - (y - minimum)/(maximum - minimum)

In [4]:
def check_violation(threshold, solution):
    # Cereals & Grains
    group1 = [1, 11, 12, 14, 15, 22, 23]
    group1_names = [list(solution.keys())[i] for i in group1]
    values1 = [solution[x] for x in group1_names]
    food_in_group1 = sum(values1)*100
#     print(f'food_in_group1 {food_in_group1}')
    idealG1 = 400
    distG1 = food_in_group1 - idealG1
    # Pulses & Vegetables
    group2 = [0, 6, 10, 13]
    group2_names = [list(solution.keys())[i] for i in group2]
    values2 = [solution[x] for x in group2_names]
    food_in_group2 = sum(values2)*100
#     print(f'food_in_group2 {food_in_group2}')
    idealG2 = 65
    distG2 = food_in_group2 - idealG2
    # Oils & Fats
    group3 = [21]
    group3_names = [list(solution.keys())[i] for i in group3]
    values3 = [solution[x] for x in group3_names]
    food_in_group3 = sum(values3)*100
#     print(f'food_in_group3 {food_in_group3}')
    idealG3 = 27
    distG3 = food_in_group3 - idealG3
    # Mixed & Blended Foods
    group4 = [5, 24, 16, 17, 18, 19]
    group4_names = [list(solution.keys())[i] for i in group4]
    values4 = [solution[x] for x in group4_names]
    food_in_group4 = sum(values4)*100
#     print(f'food_in_group4 {food_in_group4}')
    idealG4 = 45
    distG4 = food_in_group4 - idealG4
    # Meat & Fish & Dairy
    group5 = [2, 3, 4, 7, 8]
    group5_names = [list(solution.keys())[i] for i in group5]
    values5 = [solution[x] for x in group5_names]
    food_in_group5 = sum(values5)*100
#     print(f'food_in_group5 {food_in_group5}')
    idealG5 = 30
    distG5 = food_in_group5 - idealG5
    real_palatability = np.round(math.sqrt(distG1 ** 2 + (5.7 * distG2) ** 2 + (16.6 * distG3) ** 2
                                      + (4.4 * distG4) ** 2 + (6.6 * distG5) ** 2), 3)
    real_palatability_norm = normalize(real_palatability)
    return 1-int(real_palatability_norm>=threshold), real_palatability_norm

## Define the conceptual model

In [5]:
def init_conceptual_model(cost_p):
    N = list(nutr_val.index)  # foods
    M = nutr_req.columns  # nutrient requirements
    model = ConcreteModel('TPDP')
    '''
    Decision variables
    '''
    model.x = Var(N, domain=NonNegativeReals)  # variables controlling the food basket
    '''
    Objective function.
    '''
    def obj_function(model):
        return sum(cost_p[food].item()*model.x[food] for food in N)
    model.OBJ = Objective(rule=obj_function, sense=minimize)
    '''
    Nutrients requirements constraint.
    '''
    def constraint_rule1(model, req):
        return sum(model.x[food] * nutr_val.loc[food, req] for food in N) >= nutr_req[req].item()
    model.Constraint1 = Constraint(M, rule=constraint_rule1)
    '''
    Sugar constraint
    '''
    def constraint_rule2(model):
        return model.x['Sugar'] == 0.2
    model.Constraint2 = Constraint(rule=constraint_rule2)
    '''
    Salt constraint
    '''
    def constraint_rule3(model):
        return model.x['Salt'] == 0.05
    model.Constraint3 = Constraint(rule=constraint_rule3)
    return model

## Learn the *palatability* constraints

### Data Loading

In [6]:
nutr_val = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_val', index_col='Food')
nutr_req = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='nutr_req', index_col='Type')
cost_p = pd.read_excel('processed-data/Syria_instance.xlsx', sheet_name='FoodCost', index_col='Supplier').iloc[0,:]
dataset = pd.read_csv('processed-data/WFP_dataset.csv').sample(frac=1)

y = dataset['label']
X = dataset.drop(['label'], axis=1, inplace=False)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

### Training

In [7]:
alg = 'cart'
viol_rule = 0.5

# ### Example with grouping/bootstrap
# gr=True
# bs = 2

### Example with single model
gr=False
bs = 0

alg_list = [alg]

print("Algorithm = %s" % alg)
print("Bootstrap iterations = %d" % bs)
print("Violation rule = %s" % str(viol_rule))
code_version = 'AAAI-23_example'

version = 'TPDP_robust'
outcome = 'palatability'
threshold = 0.5

Algorithm = cart
Bootstrap iterations = 0
Violation rule = 0.5


In [8]:
data = X_train
outcome_list = {'palatability': {'lb':threshold, 'ub':None, 'objective_weight':0,'group_models':gr,
'task_type': 'continuous', 'alg_list':alg_list, 'bootstrap_iterations':bs,
                                   'X_train':X_train, 'y_train':y_train, 'X_test':X_test, 'y_test':y_test,
                                   'dataset_path':'processed-data/WFP_dataset.csv'}}

performance = opticl.train_ml_models(outcome_list, version)

if not os.path.exists('results'):
    os.makedirs('results')
performance.to_csv('results/%s/%s_performance.csv' % (alg, code_version))

columns_df = ['algorithm','iteration','price_matrix']+list(X.columns)+['objective_function', 'real_palat', 'pred_palat', 'violation', 'time']

print("\nPreparing model master")
if viol_rule == 'average':
    gr_method = 'average'
    max_viol = None
    print("Group method = %s" % (gr_method))
    gr_string = 'average'
else: 
    gr_method = 'violation'
    max_viol = float(viol_rule)
    print("Group method = %s (violation limit = %.2f)" % (gr_method, max_viol))
    gr_string = 'violation_%.2f' % max_viol

Learning a model for palatability
No bootstrap - training on full training data
training palatability with cart
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = cart, metric = None
saving... results/cart_palatability_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train MSE: 0.010676479379551226
Train R2: 0.7908556050356943
-------------------testing evaluation-----------------------
Test MSE: 0.015873275949482357
Test R2: 0.6909820618521263

Saving the performance...
Done!

Preparing model master
Group method = violation (violation limit = 0.50)


### Select fitted models

In [9]:
mm = opticl.initialize_model_master(outcome_list)
mm.loc[outcome,'group_method'] = gr_method
mm.loc[outcome,'max_violation'] = max_viol
model_master = opticl.model_selection(mm, performance)

if not os.path.exists('experiments'):
    print('Creating folder...')
    os.makedirs('experiments')
model_master.to_csv('experiments/model_master_%s.csv' % (code_version), index = True)

opticl.check_model_master(model_master)
model_master

                                                          model        task  \
palatability  {'results/cart/TPDP_robust_palatability_model....  continuous   

             objective   lb    ub  \
palatability         0  0.5  None   

                                                       features  \
palatability  Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...   

                                                   var_features  \
palatability  Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...   

             contex_features group_models group_method ensemble_weights  \
palatability              {}        False    violation             None   

             max_violation trust_region                    dataset_path  \
palatability           0.5         True  processed-data/WFP_dataset.csv   

             clustering_model enlargement SCM_counterfactuals  
palatability             None         [0]                None  
Checking model
No learned objective
1 learned constrained outcomes

,model,task,objective,lb,ub,features,var_features,contex_features,group_models,group_method,ensemble_weights,max_violation,trust_region,dataset_path,clustering_model,enlargement,SCM_counterfactuals
palatability,{'results/cart/TPDP_robust_palatability_model....,continuous,0,0.5,None,"Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...","Index(['Beans', 'Bulgur', 'Cheese', 'Fish', 'M...",{},False,violation,None,0.5,True,processed-data/WFP_dataset.csv,None,[0],None


## Solve the optimization problem

In [10]:
seed = 1
np.random.seed(seed)
price_random = pd.Series(np.random.random(len(cost_p))*1000)
price_random.index = cost_p.index

start_cm = time.time()
conceptual_model= init_conceptual_model(price_random)
cm = time.time() - start_cm

start_opticl = time.time()
final_model = opticl.optimization_MIP(conceptual_model, model_master)
model_opticl = time.time() - start_opticl
# final_model.write('experiments/mip_%s_seed_%d.lp' % (code_version, seed))
opt = SolverFactory('glpk')
start_time = time.time()
results = opt.solve(final_model) 
computation_time = time.time() - start_time

Generating constraints for the trust region using 5000 samples.
... Trust region defined.
Embedding constraints for palatability
Adding single model.


## Evaluation

In [11]:
pred_palat = value(final_model.y[outcome])
violation_bool, real_palat = check_violation(threshold,  final_model.x.get_values())
## Save solutions
solution = final_model.x.get_values()

if gr:
    final_model.GroupAvgpalatability.pprint()
    if gr_method == 'violation':
        final_model.constraintViolpalatability.pprint()

print('\n################################### Summary ###################################')
print(f'Algorithm used to learn the constraints: {alg}')
print("Total cost: %.3f $" % value(final_model.OBJ))
print("Predicted palatability: %.3f" % pred_palat)
print("Real palatability: %.3f" % real_palat)

print(f'\nConceptual model defined in {np.round(cm, 3)} seconds')
if model_master.loc['palatability', 'trust_region']:
    print(f'Learned constraints & trust region embedded in {np.round(model_opticl, 3)} seconds')
else:
    print(f'Learned constraints embedded in {np.round(model_opticl, 3)} seconds')
print(f'Problem Solved in {np.round(computation_time, 3)} seconds')
print('###############################################################################')


################################### Summary ###################################
Algorithm used to learn the constraints: cart
Total cost: 1007.882 $
Predicted palatability: 0.512
Real palatability: 0.538

Conceptual model defined in 0.006 seconds
Learned constraints & trust region embedded in 1.431 seconds
Problem Solved in 0.651 seconds
###############################################################################
